# SHPB Re-Analysis Utility Example

This notebook demonstrates how to use the `SHPBReanalyzer` class to re-run SHPB analysis when parameters change.

**Two modes are supported:**
1. **Analysis-only mode** (fast): Re-run stress-strain calculation using existing aligned pulses
2. **Full re-alignment mode**: Re-run from raw data with updated alignment parameters

**Key insight**: The processed CSV contains aligned pulses for fast re-analysis, while the raw CSV + TTL detection parameters allow full re-alignment when needed.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
from pathlib import Path

# Add project root to path
proj_root = Path.cwd().parent.parent
sys.path.insert(0, str(proj_root))

# Import reanalysis utility
from dynamat.mechanical.shpb.utils import SHPBReanalyzer

# Import ontology manager
from dynamat.ontology import OntologyManager

print("Imports successful!")

## Initialize Ontology Manager and Reanalyzer

In [ ]:
# Create ontology manager
manager = OntologyManager()

# Create reanalyzer instance
reanalyzer = SHPBReanalyzer(manager)

print(f"Reanalyzer initialized: {reanalyzer}")

## Load Existing Test

Load an existing SHPB test TTL file. This will:
- Parse the TTL to extract equipment URIs and parameters
- Load the processed CSV containing aligned pulses
- Load the raw CSV for full re-alignment mode
- Query equipment properties from the ontology

In [ ]:
# Define path to test TTL
test_path = "user_data/specimens/DYNML-SS316A356-0050/DYNML_SS316A356_0050_SHPBTest.ttl"

# Load the test
reanalyzer.load_test(test_path)

print(f"Test loaded: {reanalyzer}")

## Inspect Current Parameters

View the parameters that were extracted from the test and ontology.

In [ ]:
# Get current parameters
params = reanalyzer.get_current_parameters()

print("=" * 60)
print("INCIDENT BAR PROPERTIES")
print("=" * 60)
for key, value in params['incident_bar'].items():
    print(f"  {key}: {value}")

print("\n" + "=" * 60)
print("SPECIMEN PROPERTIES")
print("=" * 60)
for key, value in params['specimen'].items():
    print(f"  {key}: {value}")

print("\n" + "=" * 60)
print("ALIGNMENT PARAMETERS")
print("=" * 60)
alignment_params = reanalyzer.get_alignment_parameters()
for key, value in alignment_params.items():
    print(f"  {key}: {value}")

## Example 1: Analysis-Only Mode (Bar Recalibration)

This mode is fast because it uses the existing aligned pulses from the processed CSV.
Use this when you want to see how stress-strain curves change with different bar properties.

In [ ]:
# Store original wave speed for comparison
original_wave_speed = params['incident_bar']['wave_speed']
print(f"Original bar wave speed: {original_wave_speed} mm/ms")

# Update bar wave speed (e.g., after recalibration measurement)
# Increase by 1% to see the effect
new_wave_speed = original_wave_speed * 1.01
reanalyzer.update_bar_property('incident', 'wave_speed', new_wave_speed)
reanalyzer.update_bar_property('transmission', 'wave_speed', new_wave_speed)

print(f"Updated bar wave speed: {new_wave_speed} mm/ms")

# View all parameter changes
changes = reanalyzer.get_parameter_changes()
print("\nParameter changes:")
for param_path, (old_val, new_val) in changes.items():
    print(f"  {param_path}: {old_val} -> {new_val}")

In [ ]:
# Run analysis-only recalculation (fast - uses existing aligned pulses)
print("Running analysis-only recalculation...")
results_new = reanalyzer.recalculate(mode='analysis_only')

# Get metrics
metrics_new = reanalyzer.get_metrics()

print("\nRecalculation complete!")
print(f"  FBC: {metrics_new['FBC']:.4f}")
print(f"  DSUF: {metrics_new['DSUF']:.4f}")
print(f"  SEQI: {metrics_new['SEQI']:.4f}")
print(f"  SOI: {metrics_new['SOI']:.4f}")

### Compare Original vs Recalculated

Load the original results for comparison.

In [ ]:
# Load original processed data for comparison
original_csv = proj_root / "user_data/specimens/DYNML-SS316A356-0050/processed/DYNML_SS316A356_0050_SHPBTest_processed.csv"
original_df = pd.read_csv(original_csv)

# Create comparison plot
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Plot 1: Stress-Strain Curve (1-wave)
ax1 = axes[0]
ax1.plot(original_df['strain_1w'], original_df['stress_1w'], 
         lw=2, label='Original', alpha=0.8)
ax1.plot(results_new['strain_1w'], results_new['stress_1w'], 
         lw=2, ls='--', label='Recalculated (+1% wave speed)', alpha=0.8)
ax1.set_xlabel('Engineering Strain')
ax1.set_ylabel('Engineering Stress (MPa)')
ax1.set_title('Stress-Strain (1-wave)')
ax1.legend()
ax1.grid(alpha=0.3)

# Plot 2: Stress-Strain Curve (3-wave)
ax2 = axes[1]
ax2.plot(original_df['strain_3w'], original_df['stress_3w'], 
         lw=2, label='Original', alpha=0.8)
ax2.plot(results_new['strain_3w'], results_new['stress_3w'], 
         lw=2, ls='--', label='Recalculated (+1% wave speed)', alpha=0.8)
ax2.set_xlabel('Engineering Strain')
ax2.set_ylabel('Engineering Stress (MPa)')
ax2.set_title('Stress-Strain (3-wave)')
ax2.legend()
ax2.grid(alpha=0.3)

# Plot 3: Strain Rate vs Time
ax3 = axes[2]
ax3.plot(original_df['time'], original_df['strain_rate_1w'], 
         lw=2, label='Original', alpha=0.8)
ax3.plot(results_new['time'], results_new['strain_rate_1w'], 
         lw=2, ls='--', label='Recalculated', alpha=0.8)
ax3.set_xlabel('Time (ms)')
ax3.set_ylabel('Strain Rate (1/s)')
ax3.set_title('Strain Rate vs Time')
ax3.legend()
ax3.grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate percentage difference in peak stress
orig_peak = original_df['stress_1w'].max()
new_peak = results_new['stress_1w'].max()
pct_diff = 100 * (new_peak - orig_peak) / orig_peak
print(f"\nPeak stress comparison:")
print(f"  Original: {orig_peak:.2f} MPa")
print(f"  Recalculated: {new_peak:.2f} MPa")
print(f"  Difference: {pct_diff:+.2f}%")

### Save Recalculated Results

Save the recalculated results with a version suffix.

In [ ]:
# Save with version suffix (creates new files, doesn't overwrite original)
csv_path, ttl_path = reanalyzer.save(version_suffix="_recalibrated")

print("Files saved:")
print(f"  CSV: {csv_path}")
print(f"  TTL: {ttl_path}")

## Example 2: Full Re-Alignment Mode

This mode re-runs the entire analysis pipeline from raw data, including pulse detection and alignment.
Use this when you want to change alignment parameters like search bounds or k_linear.

In [ ]:
# Create a fresh reanalyzer instance
reanalyzer2 = SHPBReanalyzer(manager)
reanalyzer2.load_test(test_path)

# View current alignment parameters
align_params = reanalyzer2.get_alignment_parameters()
print("Current alignment parameters:")
for key, value in align_params.items():
    print(f"  {key}: {value}")

In [ ]:
# Update alignment parameters
# Widen the search bounds for transmitted pulse
reanalyzer2.update_alignment_param('search_bounds_t', (3100, 3400))

# Adjust k_linear for different linear region definition
reanalyzer2.update_alignment_param('k_linear', 0.40)

print("Updated alignment parameters:")
for key, value in reanalyzer2.get_alignment_parameters().items():
    print(f"  {key}: {value}")

In [ ]:
# Run full re-alignment (slower - re-runs from raw data)
print("Running full re-alignment...")
print("(This may take a moment as it re-runs pulse detection and alignment)")

results_full = reanalyzer2.recalculate(mode='full')

# Get metrics
metrics_full = reanalyzer2.get_metrics()

print("\nFull recalculation complete!")
print(f"  FBC: {metrics_full['FBC']:.4f}")
print(f"  DSUF: {metrics_full['DSUF']:.4f}")
print(f"  SEQI: {metrics_full['SEQI']:.4f}")
print(f"  SOI: {metrics_full['SOI']:.4f}")

# Show new alignment shifts
new_align = reanalyzer2.get_alignment_parameters()
print(f"\nNew alignment shifts:")
print(f"  Transmitted shift: {new_align['shift_t']}")
print(f"  Reflected shift: {new_align['shift_r']}")

In [ ]:
# Compare aligned pulses
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Aligned pulses comparison
ax1 = axes[0]
ax1.plot(original_df['time'], original_df['incident'], lw=2, label='Incident (Original)', alpha=0.6)
ax1.plot(original_df['time'], original_df['transmitted'], lw=2, label='Transmitted (Original)', alpha=0.6)
ax1.plot(results_full['time'], results_full['incident'], lw=2, ls='--', label='Incident (Re-aligned)', alpha=0.8)
ax1.plot(results_full['time'], results_full['transmitted'], lw=2, ls='--', label='Transmitted (Re-aligned)', alpha=0.8)
ax1.set_xlabel('Time (ms)')
ax1.set_ylabel('Strain (normalized)')
ax1.set_title('Aligned Pulses Comparison')
ax1.legend()
ax1.grid(alpha=0.3)

# Plot 2: Stress-strain comparison
ax2 = axes[1]
ax2.plot(original_df['strain_1w'], original_df['stress_1w'], lw=2, label='Original', alpha=0.8)
ax2.plot(results_full['strain_1w'], results_full['stress_1w'], lw=2, ls='--', label='Re-aligned', alpha=0.8)
ax2.set_xlabel('Engineering Strain')
ax2.set_ylabel('Engineering Stress (MPa)')
ax2.set_title('Stress-Strain After Re-alignment')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

## Example 3: Parameter Sensitivity Study

Use the reanalyzer to study how stress-strain curves change with different parameters.

In [ ]:
# Study effect of wave speed variation
wave_speed_variations = [0.98, 0.99, 1.00, 1.01, 1.02]  # -2% to +2%
results_by_variation = {}

# Get original wave speed
reanalyzer3 = SHPBReanalyzer(manager)
reanalyzer3.load_test(test_path)
original_ws = reanalyzer3.get_current_parameters()['incident_bar']['wave_speed']

print("Running sensitivity study for wave speed...")
for factor in wave_speed_variations:
    # Reset reanalyzer
    reanalyzer3 = SHPBReanalyzer(manager)
    reanalyzer3.load_test(test_path)
    
    # Update wave speed
    new_ws = original_ws * factor
    reanalyzer3.update_bar_property('incident', 'wave_speed', new_ws)
    reanalyzer3.update_bar_property('transmission', 'wave_speed', new_ws)
    
    # Calculate
    results = reanalyzer3.recalculate(mode='analysis_only')
    results_by_variation[factor] = results
    
    pct_change = (factor - 1) * 100
    print(f"  Wave speed {pct_change:+.0f}%: Peak stress = {results['stress_1w'].max():.2f} MPa")

print("Sensitivity study complete!")

In [ ]:
# Plot sensitivity results
fig, ax = plt.subplots(figsize=(10, 6))

colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(wave_speed_variations)))

for (factor, results), color in zip(results_by_variation.items(), colors):
    pct_change = (factor - 1) * 100
    label = f"Wave speed {pct_change:+.0f}%"
    ax.plot(results['strain_1w'], results['stress_1w'], 
            lw=2, label=label, color=color, alpha=0.8)

ax.set_xlabel('Engineering Strain', fontsize=12)
ax.set_ylabel('Engineering Stress (MPa)', fontsize=12)
ax.set_title('Wave Speed Sensitivity Study', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Summary table
print("\nSensitivity Summary:")
print(f"{'Wave Speed Change':>18} | {'Peak Stress (MPa)':>18} | {'Peak Strain':>12}")
print("-" * 55)
for factor, results in results_by_variation.items():
    pct_change = (factor - 1) * 100
    peak_stress = results['stress_1w'].max()
    peak_strain = results['strain_1w'].max()
    print(f"{pct_change:+18.0f}% | {peak_stress:18.2f} | {peak_strain:12.4f}")

## Summary

The `SHPBReanalyzer` provides a convenient way to:

1. **Load existing tests**: Parse TTL and CSV files, extract parameters from ontology
2. **Update parameters**: Change bar, specimen, gauge, or alignment parameters
3. **Recalculate**: Run analysis-only (fast) or full re-alignment modes
4. **Save results**: Create versioned output files or overwrite originals

**Use cases:**
- Bar recalibration (updated wave speed or elastic modulus)
- Specimen dimension corrections
- Alignment parameter optimization
- Parameter sensitivity studies
- What-if analysis for equipment changes